<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#一、持有A股流通市值" data-toc-modified-id="一、持有A股流通市值-1">一、持有A股流通市值</a></span></li><li><span><a href="#二、体量" data-toc-modified-id="二、体量-2">二、体量</a></span></li><li><span><a href="#三、仓位" data-toc-modified-id="三、仓位-3">三、仓位</a></span></li><li><span><a href="#四、行业" data-toc-modified-id="四、行业-4">四、行业</a></span></li></ul></div>

In [1]:
from it_data_tool.data_tool import *
from lowrisk_it_tools.WindPy import *
from macrotoolchain.mainAPI import *
di.display_html(getCatalogHtml(),raw=True)
from datetime import datetime,timedelta
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
di.HTML(getCatalogHtml())
h1Title('私募基金投资行为')
h2Title('更新时间： {}年{}月{}日'.format(Date.today().year, Date.today().month, Date.today().day))

私募基金投资行为

# 一、持有A股流通市值

计算方法：2015-06-30之前，采用中国A股前十大流通股东中股东名称包含“私募”或“资金信托计划”，持有数量\*当日收盘价，除以0.05为全部持股市值；2015-06-30之后，采用私募基金管理机构:实缴规模:私募证券投资基金\*仓位估计持股市值，2015、2017年仓位为0.8，2021-12-31仓位为0.75，2022-03-31仓位为0.65，其余时间仓位为0.6

In [2]:
# 所用日期序列
start_date = '20051231'
end_date = datetime.today().strftime('%Y%m%d')
date_list_base = pd.date_range(start_date,end_date,freq='Q')
date_list = [d.strftime('%Y%m%d') for d in date_list_base]

In [3]:
# A股流通市值
def cal_a_flow_nav():
    sql = '''
        select S_INFO_WINDCODE as scode, TRADE_DT, S_DQ_MV/1e4 as flow_market_value
        from ads_wind_AShareEODDerivativeIndicator_a
        where TRADE_DT in {}
        order by TRADE_DT, S_INFO_WINDCODE
        '''.format(tuple(date_list))
    df = get_quant_db_hz(sql).groupby(['trade_dt'])['flow_market_value'].sum()
    df.index = pd.to_datetime(df.index)
    return df

a_flow_nav = cal_a_flow_nav()

In [4]:
# 前10大流通股东，亿股
sql = '''
    select s_info_windcode as scode, s_holder_enddate as trade_dt, s_holder_holdercategory as type, s_holder_name as holder, s_holder_quantity/1e8 as shares
    from ads_wind_asharefloatholder_a
    where s_holder_enddate in {}
    order by s_holder_enddate,s_info_windcode
    '''.format(tuple(date_list))
float_holders = get_quant_db_hz(sql)

# A股收盘价
trade_date_list = w.tdays("2005-10-31", datetime.today().strftime('%Y%m%d'), "Period=Q",usedf=True)[1].index.tolist()
trade_date_list = [x.strftime('%Y%m%d') for x in trade_date_list]
sql = '''
    select S_INFO_WINDCODE as scode, TRADE_DT as old_dt, S_DQ_CLOSE as price
    from ads_wind_ashareeodprices_a
    where TRADE_DT in {}
    order by TRADE_DT, S_INFO_WINDCODE
    '''.format(tuple(trade_date_list))
astock_price = get_quant_db_hz(sql)
# 处理日期
date_df = pd.DataFrame(columns=['old_dt','trade_dt'])
if len(trade_date_list)!=len(date_list):
    trade_date_list = trade_date_list[:-1]
date_df['old_dt'] = trade_date_list
date_df['trade_dt'] = date_list
astock_price = pd.merge(astock_price,date_df,on=['old_dt']).drop(['old_dt'],axis=1)

In [20]:
# 私募
def cal_simu_nav():
    # 找私募基金，先这样吧
    use_holders = float_holders[(float_holders.holder.str.contains('私募')) | (float_holders.holder.str.contains('资金信托计划'))]
    # 私募基金管理机构:实缴规模:私募证券投资基金
    df = w.edb("M5524611,M5543215", start_date, end_date,usedf=True)[1]
    df.columns = ['simu_market_value','tmp']
    df['simu_market_value'] = df['simu_market_value'].fillna(df['tmp'])
    df.index = pd.to_datetime(df.index)
    df = df.loc[date_list_base,:].drop(['tmp'],axis=1)
    use_holders = pd.merge(use_holders,astock_price,on=['scode','trade_dt'])
    use_holders['test'] = use_holders['shares']*use_holders['price']
    use_holders = use_holders.groupby(['trade_dt'])['test'].sum()
    use_holders.index = pd.to_datetime(use_holders.index)
    df = pd.concat([df,use_holders],axis=1)
    df['fill'] = df['test']/0.05/0.6
    df['simu_market_value'] = df['simu_market_value'].fillna(df['fill'])
    df['times'] = 0.6
    df.loc[(df.index>='2015') & (df.index<='2016'),'times'] = 0.8
    df.loc[(df.index>='2017') & (df.index<='2018'),'times'] = 0.8
    df.loc[(df.index=='2021-12-31'),'times'] = 0.75
    df.loc[(df.index=='2022-03-31'),'times'] = 0.65
    df['simu_market_value'] *= df['times']
    return df['simu_market_value']
    
simu_nav = cal_simu_nav()

In [6]:
nav = pd.concat([simu_nav,a_flow_nav],axis=1)
nav['bili'] = nav['simu_market_value']/nav['flow_market_value']*100

In [7]:
from lowrisk.CustomECharts import *

s = Data(timeList=nav.index,dataList=nav['simu_market_value'],title='私募基金持有A股流通市值（亿元）')
bar_gra = barGraph(s,color='palevioletred',stack='1')
s = Data(timeList=nav.index,dataList=nav['bili'],title='占A股总流通市值之比（%）')
line_gra = lineGraph(s,color='midnightblue',yAxisIndex=1)
plot_arr = [bar_gra,line_gra]

goechart(plot_arr,setLayout(xAxis = setAxis(type="time"),
                            yAxis = [setAxis(),setAxis()],
                            title=setTitle("私募基金持有A股流通市值"),
                            legend=dict(
                                        x="center",
                                        y="bottom",
                                        orient='horizontal',
                                        padding=[0,50,0,0]
                                        ),
                            grid=[{
                                    "left":"5%",
                                    "right":"10%",
                                    "height":"70%"
                                }],
                            dataZoom=[
                                        {
                                            "type":"slider",
                                            "xAxisIndex":0,
                                            "f'selector':TrueilterMode":'empty',
                                            "bottom":"8%"
                                        },
                                        {
                                            "type":"slider",
                                            "yAxisIndex":0,
                                            "filterMode":'empty'
                                        },
                                    ]
                           )
        )

# 二、体量

计算方法：私募基金管理机构:实缴规模:私募证券投资基金

In [8]:
hedge_fund_aum_df = w.edb("M5524611,M5543215", start_date, end_date,usedf=True)[1]
hedge_fund_aum_df.columns = ['私募证券基金管理规模（季度）','私募证券基金管理规模（月度）']
hedge_fund_aum_df.index = pd.to_datetime(hedge_fund_aum_df.index)
hedge_fund_aum_df.index = [i.strftime("%Y%m%d") for i in hedge_fund_aum_df.index]

In [9]:
from lowrisk.CustomECharts import *
s1 = Data(timeList= [Date.fromString(date = i, format = '%Y%m%d') for i in hedge_fund_aum_df.dropna().index], dataList=hedge_fund_aum_df.dropna()['私募证券基金管理规模（季度）'] ,title='私募证券基金管理规模（季度）') 
s2 = Data(timeList= [Date.fromString(date = i, format = '%Y%m%d') for i in hedge_fund_aum_df.dropna().index], dataList=hedge_fund_aum_df.dropna()['私募证券基金管理规模（季度）'].pct_change()*100 ,title='私募证券基金管理规模（季度）-环比') 
plot_arr = [barGraph(s2,color='palevioletred'),lineGraph(s1,yAxisIndex=1,color='midnightblue')]
s1 = Data(timeList= [Date.fromString(date = i, format = '%Y%m%d') for i in hedge_fund_aum_df.index], dataList=hedge_fund_aum_df['私募证券基金管理规模（月度）'] ,title='私募证券基金管理规模（月度）') 
s2 = Data(timeList= [Date.fromString(date = i, format = '%Y%m%d') for i in hedge_fund_aum_df.index], dataList=hedge_fund_aum_df['私募证券基金管理规模（月度）'].pct_change()*100 ,title='私募证券基金管理规模（月度）-环比') 
plot_arr += [barGraph(s2,color='pink'),lineGraph(s1,yAxisIndex=1,color='skyblue')]

goechart(plot_arr,
         setLayout(xAxis = setAxis(type="time"),
                   yAxis=[
                            setAxis(title='环比（%）'),
                            setAxis(title='规模（亿元）'),
                         ],
                   title=setTitle("私募证券基金管理规模（亿元）"),
                   legend=dict(
                                        x="center",
                                        y="bottom",
                                        orient='horizontal',
                                        padding=[0,50,0,0]
                                        ),
                            grid=[{
                                    "left":"5%",
                                    "right":"10%",
                                    "height":"70%"
                                }],
                            dataZoom=[
                                        {
                                            "type":"slider",
                                            "xAxisIndex":0,
                                            "f'selector':TrueilterMode":'empty',
                                            "bottom":"8%"
                                        },
                                        {
                                            "type":"slider",
                                            "yAxisIndex":0,
                                            "filterMode":'empty'
                                        },
                                    ]
                           )
        )

# 三、仓位

计算方法：每周跟进中信研究员内部数据，统计范围为在中信托管的私募产品

In [10]:
# 中信调研数据
dycw = pd.read_excel('私募仓位.xlsx',index_col=0).rename(columns={'仓位':'私募仓位'})
dycw['私募仓位'] *= 100
jxcw = pd.read_excel('私募仓位-解析.xlsx',index_col=0)
jxcw = jxcw[jxcw.index<='2022-04']
dycw = pd.concat([jxcw,dycw],axis=0)
dycw.index = pd.to_datetime(dycw.index)

In [11]:
from lowrisk.CustomECharts import *
s = Data(timeList=dycw.index,dataList=dycw['私募仓位'].tolist(),title='私募仓位%')
line_gra = lineGraph(s,color='midnightblue')
plot_arr = [line_gra]
goechart(plot_arr,setLayout(xAxis = setAxis(type="time"),
                            yAxis = setAxis(range=[50,100]),
                            title=setTitle("中信渠道调研私募仓位"),
                            legend=dict(
                                        x="center",
                                        y="bottom",
                                        orient='horizontal',
                                        padding=[0,50,0,0]
                                        ),
                            grid=[{
                                    "left":"5%",
                                    "right":"10%",
                                    "height":"70%"
                                }],
                            dataZoom=[
                                        {
                                            "type":"slider",
                                            "xAxisIndex":0,
                                            "f'selector':TrueilterMode":'empty',
                                            "bottom":"8%"
                                        },
                                        {
                                            "type":"slider",
                                            "yAxisIndex":0,
                                            "filterMode":'empty'
                                        },
                                    ]
                           )
        )

# 四、行业

计算方法：统计代表性私募管理人产品持股，在A股季报前十大流通股东中出现的股票，统计其行业分布

In [12]:
def cal_hangye_bili(name):
    use_holders = float_holders[(float_holders.holder.str.contains(name)) & (float_holders['type']=='2')]
    use_holders = pd.merge(use_holders,astock_price,on=['scode','trade_dt'])
    use_holders['nav'] = use_holders['shares']*use_holders['price']
    sql = '''
        select wind_code as scode, citic_industry as industry
        from ads_wind_ahshareciticsindustries_style_a
        '''
    ind = get_quant_db_hz(sql)
    df = pd.merge(use_holders,ind,on=['scode'],how='left')
    df['industry'] = df['industry'].fillna('综合')
    df.trade_dt = pd.to_datetime(df.trade_dt)
    # 行业和中信一级行业对齐
    df.loc[df.industry=='保险Ⅱ','industry'] = '非银行金融'
    df.loc[df.industry=='证券Ⅱ','industry'] = '非银行金融'
    df.loc[df.industry=='多元金融','industry'] = '非银行金融'
    df.loc[df.industry=='酒类','industry'] = '食品饮料'
    df.loc[df.industry=='食品','industry'] = '食品饮料'
    df.loc[df.industry=='饮料','industry'] = '食品饮料'
    df = df.sort_values(['trade_dt','industry']).groupby(['trade_dt','industry'])['nav'].sum().reset_index()
    df = df.assign(tot = lambda x:x.groupby(['trade_dt'])['nav'].transform('sum'),
                  w = lambda x:x['nav']/x['tot'])
    df = df.pivot_table(index=['trade_dt'],columns=['industry'],values=['w'])['w']
    df *= 100
    return df.dropna(axis=1,how='all')

gaoyi = cal_hangye_bili('高毅')
danshuiquan = cal_hangye_bili('淡水泉')
jinglin = cal_hangye_bili('景林')
yuanlesheng = cal_hangye_bili('源乐晟')
chongyang = cal_hangye_bili('重阳')

In [13]:
from lowrisk.CustomECharts import *
colors = ['darkred','crimson','indianred','darkorange','salmon','goldenrod','gold','navajowhite','lime','lightgreen',
          'forestgreen','cyan','mediumaquamarine','turquoise','lightsteelblue','lightskyblue','dodgerblue',
          'slateblue','darkblue','purple','violet','plum','lightpink','hotpink','mediumvioletred','gray','black']
def draw(df,name):
    plot_arr = []
    for i in range(len(df.columns)):
        c = df.columns[i]
        s = Data(timeList=df.index,dataList=df[c].tolist(),title=c)
        plot_arr.append(barGraph(s,stack=1,color=colors[i]))
    g = goechart(plot_arr,setLayout(xAxis = setAxis(type="time"),
                                    yAxis = setAxis(),
                                    title=setTitle("前十大流通股东口径-{}行业持仓比例（%）".format(name)),
                                    legend=dict(
                                                x="center",
                                                y="bottom",
                                                orient='horizontal',
                                                padding=[0,50,0,0]
                                                ),
                                    grid=[{
                                            "left":"5%",
                                            "right":"10%",
                                            "height":"70%"
                                        }],
                                    dataZoom=[
                                                {
                                                    "type":"slider",
                                                    "xAxisIndex":0,
                                                    "f'selector':TrueilterMode":'empty',
                                                    "bottom":"8%"
                                                },
                                                {
                                                    "type":"slider",
                                                    "yAxisIndex":0,
                                                    "filterMode":'empty'
                                                },
                                            ]
                                   )
                )
    return g

In [14]:
draw(gaoyi,'高毅')

In [15]:
draw(danshuiquan,'淡水泉')

In [16]:
draw(jinglin,'景林')

In [17]:
draw(yuanlesheng,'源乐晟')

In [18]:
draw(chongyang,'重阳')